# Description

(Please, take a look at the README.md file in this directory for instructions on how to run this notebook)

This notebook compiles information about the GWAS and TWAS for a particular cohort. For example, the set of GWAS variants, variance of predicted expression of genes, etc.

It has specicfic parameters for papermill (see under `Settings` below).

This notebook is not directly run. See README.md.

# Modules

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path
import pickle

import numpy as np
import pandas as pd

import conf
from entity import Gene

# Settings

In [3]:
# a cohort name (it could be something like UK_BIOBANK, etc)
COHORT_NAME = None

# reference panel such as 1000G or GTEX_V8
REFERENCE_PANEL = None

# predictions models such as MASHR or ELASTIC_NET
EQTL_MODEL = None

# a string with a path pointing to an imputed GWAS
GWAS_FILE = None

# a string with a path pointing where S-PrediXcan results (tissue-specific are located
SPREDIXCAN_FOLDER = None

# an f-string with one placeholder {tissue}
SPREDIXCAN_FILE_PATTERN = None

# a string with a path pointing to an S-MultiXcan result
SMULTIXCAN_FILE = None

In [4]:
# Parameters
COHORT_NAME = "emerge"
REFERENCE_PANEL = "GTEX_V8"
EQTL_MODEL = "MASHR"
GWAS_FILE = "/project/ritchie20/projects/phenoplier/base/data/emerge/gwas/EUR_411.3_imputed.txt.gz"
SPREDIXCAN_FOLDER = (
    "/project/ritchie20/projects/phenoplier/base/data/emerge/spredixcan/EUR_411.3/"
)
SPREDIXCAN_FILE_PATTERN = "eMERGE_III_spredixcan_mashr_eqtl_EUR_411.3_{tissue}.csv"
SMULTIXCAN_FILE = "/project/ritchie20/projects/phenoplier/base/data/emerge/smultixcan/eMERGE_III_smultixcan_mashr_eqtl_EUR_411.3.txt"


In [5]:
assert COHORT_NAME is not None and len(COHORT_NAME) > 0, "A cohort name must be given"

COHORT_NAME = COHORT_NAME.lower()
display(f"Cohort name: {COHORT_NAME}")

'Cohort name: emerge'

In [6]:
assert (
    REFERENCE_PANEL is not None and len(REFERENCE_PANEL) > 0
), "A reference panel must be given"

display(f"Reference panel: {REFERENCE_PANEL}")

'Reference panel: GTEX_V8'

In [7]:
assert GWAS_FILE is not None and len(GWAS_FILE) > 0, "A GWAS file path must be given"
GWAS_FILE = Path(GWAS_FILE).resolve()
assert GWAS_FILE.exists(), "GWAS file does not exist"

display(f"GWAS file path: {str(GWAS_FILE)}")

'GWAS file path: /project/ritchie20/projects/phenoplier/base/data/emerge/gwas/EUR_411.3_imputed.txt.gz'

In [8]:
assert (
    SPREDIXCAN_FOLDER is not None and len(SPREDIXCAN_FOLDER) > 0
), "An S-PrediXcan folder path must be given"
SPREDIXCAN_FOLDER = Path(SPREDIXCAN_FOLDER).resolve()
assert SPREDIXCAN_FOLDER.exists(), "S-PrediXcan folder does not exist"

display(f"S-PrediXcan folder path: {str(SPREDIXCAN_FOLDER)}")

'S-PrediXcan folder path: /project/ritchie20/projects/phenoplier/base/data/emerge/spredixcan/EUR_411.3'

In [9]:
assert (
    SPREDIXCAN_FILE_PATTERN is not None and len(SPREDIXCAN_FILE_PATTERN) > 0
), "An S-PrediXcan file pattern must be given"
assert (
    "{tissue}" in SPREDIXCAN_FILE_PATTERN
), "S-PrediXcan file pattern must have a '{tissue}' placeholder"

display(f"S-PrediXcan file template: {SPREDIXCAN_FILE_PATTERN}")

'S-PrediXcan file template: eMERGE_III_spredixcan_mashr_eqtl_EUR_411.3_{tissue}.csv'

In [10]:
assert (
    SMULTIXCAN_FILE is not None and len(SMULTIXCAN_FILE) > 0
), "An S-MultiXcan result file path must be given"
SMULTIXCAN_FILE = Path(SMULTIXCAN_FILE).resolve()
assert SMULTIXCAN_FILE.exists(), "S-MultiXcan result file does not exist"

display(f"S-MultiXcan file path: {str(SMULTIXCAN_FILE)}")

'S-MultiXcan file path: /project/ritchie20/projects/phenoplier/base/data/emerge/smultixcan/eMERGE_III_smultixcan_mashr_eqtl_EUR_411.3.txt'

In [11]:
assert (
    EQTL_MODEL is not None and len(EQTL_MODEL) > 0
), "A prediction/eQTL model must be given"

display(f"eQTL model: {EQTL_MODEL}")

'eQTL model: MASHR'

In [12]:
OUTPUT_DIR_BASE = (
    conf.RESULTS["GLS"]
    / "gene_corrs"
    / "cohorts"
    / COHORT_NAME
    / REFERENCE_PANEL.lower()
    / EQTL_MODEL.lower()
)

OUTPUT_DIR_BASE.mkdir(parents=True, exist_ok=True)

display(f"Using output dir base: {OUTPUT_DIR_BASE}")

'Using output dir base: /project/ritchie20/projects/phenoplier/base/results/gls/gene_corrs/cohorts/emerge/gtex_v8/mashr'

# Load MultiPLIER Z genes

In [13]:
multiplier_z_genes = pd.read_pickle(
    conf.MULTIPLIER["MODEL_Z_MATRIX_FILE"]
).index.tolist()

In [14]:
len(multiplier_z_genes)

6750

In [15]:
assert len(multiplier_z_genes) == len(set(multiplier_z_genes))

In [16]:
multiplier_z_genes[:5]

['GAS6', 'MMP14', 'DSP', 'MARCKSL1', 'SPARC']

# GWAS

In [17]:
gwas_file_columns = pd.read_csv(GWAS_FILE, sep="\t", nrows=2).columns
assert (
    "panel_variant_id" in gwas_file_columns
), "GWAS file must be final imputed one with column 'panel_variant_id'"
# FIXME: add other needed columns here

In [18]:
gwas_data = pd.read_csv(
    GWAS_FILE,
    sep="\t",
    usecols=["panel_variant_id", "pvalue", "zscore"],
)

In [19]:
gwas_data.shape

(8376512, 3)

In [20]:
gwas_data.head()

,panel_variant_id,zscore,pvalue
0,chr1_54490_G_A_b38,-0.719998,0.471526
1,chr1_87021_T_C_b38,-1.194425,0.232312
2,chr1_263722_C_G_b38,0.207573,0.835562
3,chr1_594402_C_T_b38,-0.375183,0.707524
4,chr1_630555_C_T_b38,-0.371214,0.710478


In [21]:
gwas_data.dropna().shape

(8376512, 3)

In [22]:
# remove SNPs with no results
gwas_data = gwas_data.dropna()

In [23]:
gwas_data.shape

(8376512, 3)

## Save GWAS variants

In [24]:
gwas_data.head()

,panel_variant_id,zscore,pvalue
0,chr1_54490_G_A_b38,-0.719998,0.471526
1,chr1_87021_T_C_b38,-1.194425,0.232312
2,chr1_263722_C_G_b38,0.207573,0.835562
3,chr1_594402_C_T_b38,-0.375183,0.707524
4,chr1_630555_C_T_b38,-0.371214,0.710478


In [25]:
# in eMERGE, some values here are repeated
gwas_data["panel_variant_id"].is_unique

False

In [26]:
gwas_variants_ids_set = frozenset(gwas_data["panel_variant_id"])
list(gwas_variants_ids_set)[:5]

['chr12_80716377_A_C_b38',
 'chr3_169761064_T_G_b38',
 'chr3_69336196_C_T_b38',
 'chr20_57717967_G_A_b38',
 'chr9_1221840_T_C_b38']

In [27]:
with open(OUTPUT_DIR_BASE / "gwas_variant_ids.pkl", "wb") as handle:
    pickle.dump(gwas_variants_ids_set, handle, protocol=pickle.HIGHEST_PROTOCOL)

# TWAS

## Available tissues for eQTL model

In [28]:
prediction_model_tissues = conf.PHENOMEXCAN["PREDICTION_MODELS"][
    f"{EQTL_MODEL}_TISSUES"
].split(" ")

In [29]:
len(prediction_model_tissues)

49

In [30]:
prediction_model_tissues[:5]

['Whole_Blood',
 'Brain_Hypothalamus',
 'Brain_Amygdala',
 'Brain_Nucleus_accumbens_basal_ganglia',
 'Artery_Coronary']

## S-MultiXcan results

In [31]:
smultixcan_results = pd.read_csv(
    SMULTIXCAN_FILE, sep="\t", usecols=["gene", "gene_name", "pvalue", "n", "n_indep"]
)

In [32]:
smultixcan_results.shape

(22197, 5)

In [33]:
smultixcan_results = smultixcan_results.dropna()

In [34]:
smultixcan_results.shape

(22193, 5)

In [35]:
smultixcan_results = smultixcan_results.assign(
    gene_id=smultixcan_results["gene"].apply(lambda g: g.split(".")[0])
)

In [36]:
smultixcan_results.head()

,gene,gene_name,pvalue,n,n_indep,gene_id
0,ENSG00000198670.11,LPA,7.680689e-10,9.0,6.0,ENSG00000198670
1,ENSG00000111845.4,PAK1IP1,1.332263e-05,29.0,3.0,ENSG00000111845
2,ENSG00000141569.10,TRIM65,2.200177e-05,48.0,4.0,ENSG00000141569
3,ENSG00000147883.10,CDKN2B,2.787706e-05,29.0,3.0,ENSG00000147883
4,ENSG00000106333.12,PCOLCE,3.141458e-05,44.0,3.0,ENSG00000106333


In [37]:
assert smultixcan_results["gene_id"].is_unique

### Get common genes with MultiPLIER

In [38]:
common_genes = set(multiplier_z_genes).intersection(
    set(smultixcan_results["gene_name"])
)

In [39]:
len(common_genes)

6427

In [40]:
sorted(list(common_genes))[:5]

['A2M', 'AAAS', 'AANAT', 'AARS', 'AARS2']

## Genes info

In [41]:
multiplier_gene_obj = {
    gene_name: Gene(name=gene_name)
    for gene_name in common_genes
    if gene_name in Gene.GENE_NAME_TO_ID_MAP
}

In [42]:
# delete common_genes, from now on, genes_info should be used for common genes
del common_genes

In [43]:
len(multiplier_gene_obj)

6427

In [44]:
assert multiplier_gene_obj["GAS6"].ensembl_id == "ENSG00000183087"

In [45]:
_gene_obj = list(multiplier_gene_obj.values())

genes_info = pd.DataFrame(
    {
        "name": [g.name for g in _gene_obj],
        "id": [g.ensembl_id for g in _gene_obj],
        "chr": [g.chromosome for g in _gene_obj],
        "band": [g.band for g in _gene_obj],
        "start_position": [g.get_attribute("start_position") for g in _gene_obj],
        "end_position": [g.get_attribute("end_position") for g in _gene_obj],
    }
)

In [46]:
genes_info = genes_info.assign(
    gene_length=genes_info.apply(
        lambda x: x["end_position"] - x["start_position"], axis=1
    )
)

In [47]:
genes_info.dtypes

name               object
id                 object
chr                object
band               object
start_position    float64
end_position      float64
gene_length       float64
dtype: object

In [48]:
_tmp = genes_info[genes_info.isna().any(axis=1)]
display(_tmp)
assert _tmp.shape[0] < 5

,name,id,chr,band,start_position,end_position,gene_length
1867,TMEM133,ENSG00000170647,None,None,NaN,NaN,NaN
4926,TBCE,ENSG00000116957,None,None,NaN,NaN,NaN


In [49]:
genes_info = genes_info.dropna()

In [50]:
genes_info["chr"] = genes_info["chr"].apply(pd.to_numeric, downcast="integer")
genes_info["start_position"] = genes_info["start_position"].astype(int)
genes_info["end_position"] = genes_info["end_position"].astype(int)
genes_info["gene_length"] = genes_info["gene_length"].astype(int)

In [51]:
genes_info.dtypes

name              object
id                object
chr                int64
band              object
start_position     int64
end_position       int64
gene_length        int64
dtype: object

In [52]:
assert genes_info["name"].is_unique

In [53]:
assert genes_info["id"].is_unique

In [54]:
genes_info.shape

(6425, 7)

In [55]:
genes_info.head()

,name,id,chr,band,start_position,end_position,gene_length
0,ADAMTS14,ENSG00000138316,10,10q22.1,70672506,70762441,89935
1,TXN,ENSG00000136810,9,9q31.3,110243810,110256507,12697
2,RMI1,ENSG00000178966,9,9q21.32,83980711,84004074,23363
3,DHX58,ENSG00000108771,17,17q21.2,42101404,42112714,11310
4,XRCC4,ENSG00000152422,5,5q14.2,83077498,83353787,276289


In [56]:
genes_info.sort_values("chr")

,name,id,chr,band,start_position,end_position,gene_length
4858,KCNJ10,ENSG00000177807,1,1q23.2,159998651,160070483,71832
5493,RPS27,ENSG00000177954,1,1q21.3,153990755,153992155,1400
5488,PGM1,ENSG00000079739,1,1p31.3,63593411,63660245,66834
5486,CDK11B,ENSG00000248333,1,1p36.33,1635227,1659012,23785
2769,CYB5R1,ENSG00000159348,1,1q32.1,202961873,202967275,5402
...,...,...,...,...,...,...,...
4659,PLXNB2,ENSG00000196576,22,22q13.33,50274979,50307646,32667
1127,HSCB,ENSG00000100209,22,22q12.1,28742039,28757515,15476
5364,SYN3,ENSG00000185666,22,22q12.3,32512552,33058372,545820
2266,CLDN5,ENSG00000184113,22,22q11.21,19523024,19527545,4521


### Save

In [57]:
genes_info.to_pickle(OUTPUT_DIR_BASE / "genes_info.pkl")

## S-PrediXcan results

### Load results across all tissues

In [58]:
spredixcan_result_files = {
    t: SPREDIXCAN_FOLDER / SPREDIXCAN_FILE_PATTERN.format(tissue=t)
    for t in prediction_model_tissues
}

In [59]:
assert len(spredixcan_result_files) == len(prediction_model_tissues)
display(list(spredixcan_result_files.values())[:5])

[PosixPath('/project/ritchie20/projects/phenoplier/base/data/emerge/spredixcan/EUR_411.3/eMERGE_III_spredixcan_mashr_eqtl_EUR_411.3_Whole_Blood.csv'),
 PosixPath('/project/ritchie20/projects/phenoplier/base/data/emerge/spredixcan/EUR_411.3/eMERGE_III_spredixcan_mashr_eqtl_EUR_411.3_Brain_Hypothalamus.csv'),
 PosixPath('/project/ritchie20/projects/phenoplier/base/data/emerge/spredixcan/EUR_411.3/eMERGE_III_spredixcan_mashr_eqtl_EUR_411.3_Brain_Amygdala.csv'),
 PosixPath('/project/ritchie20/projects/phenoplier/base/data/emerge/spredixcan/EUR_411.3/eMERGE_III_spredixcan_mashr_eqtl_EUR_411.3_Brain_Nucleus_accumbens_basal_ganglia.csv'),
 PosixPath('/project/ritchie20/projects/phenoplier/base/data/emerge/spredixcan/EUR_411.3/eMERGE_III_spredixcan_mashr_eqtl_EUR_411.3_Artery_Coronary.csv')]

In [60]:
# look at the structure of one result
pd.read_csv(spredixcan_result_files["Whole_Blood"]).head()

,gene,gene_name,zscore,effect_size,pvalue,var_g,pred_perf_r2,pred_perf_pval,pred_perf_qval,n_snps_used,n_snps_in_cov,n_snps_in_model,best_gwas_p,largest_weight
0,ENSG00000204381.11,LAYN,-3.982912,NaN,0.000068,0.004661,NaN,NaN,NaN,1,1,1,0.000068,0.099104
1,ENSG00000105329.9,TGFB1,-3.946805,NaN,0.000079,0.000148,NaN,NaN,NaN,1,1,1,0.000079,0.018139
2,ENSG00000078804.12,TP53INP2,-3.890546,NaN,0.000100,0.000554,NaN,NaN,NaN,1,1,1,0.000100,0.034818
3,ENSG00000204261.8,PSMB8-AS1,3.752330,NaN,0.000175,0.006429,NaN,NaN,NaN,2,2,2,0.000100,0.260087
4,ENSG00000019995.6,ZRANB1,-3.723783,NaN,0.000196,0.000020,NaN,NaN,NaN,1,1,1,0.000196,0.009755


In [61]:
assert all(f.exists() for f in spredixcan_result_files.values())

In [62]:
spredixcan_dfs = [
    pd.read_csv(
        f,
        usecols=[
            "gene",
            "zscore",
            "pvalue",
            "n_snps_used",
            "n_snps_in_model",
        ],
    )
    .dropna(subset=["gene", "zscore", "pvalue"])
    .assign(tissue=t)
    for t, f in spredixcan_result_files.items()
]

In [63]:
assert len(spredixcan_dfs) == len(prediction_model_tissues)

In [64]:
spredixcan_dfs = pd.concat(spredixcan_dfs)

In [65]:
assert spredixcan_dfs["tissue"].unique().shape[0] == len(prediction_model_tissues)

In [66]:
spredixcan_dfs.shape

(650048, 6)

In [67]:
spredixcan_dfs = spredixcan_dfs.assign(
    gene_id=spredixcan_dfs["gene"].apply(lambda g: g.split(".")[0])
)

In [68]:
spredixcan_dfs.head()

,gene,zscore,pvalue,n_snps_used,n_snps_in_model,tissue,gene_id
0,ENSG00000204381.11,-3.982912,0.000068,1,1,Whole_Blood,ENSG00000204381
1,ENSG00000105329.9,-3.946805,0.000079,1,1,Whole_Blood,ENSG00000105329
2,ENSG00000078804.12,-3.890546,0.000100,1,1,Whole_Blood,ENSG00000078804
3,ENSG00000204261.8,3.752330,0.000175,2,2,Whole_Blood,ENSG00000204261
4,ENSG00000019995.6,-3.723783,0.000196,1,1,Whole_Blood,ENSG00000019995


In [69]:
# leave only common genes
spredixcan_dfs = spredixcan_dfs[spredixcan_dfs["gene_id"].isin(set(genes_info["id"]))]

In [70]:
spredixcan_dfs.shape

(233000, 7)

### Count number of tissues available per gene

In [71]:
spredixcan_genes_n_models = spredixcan_dfs.groupby("gene_id")["tissue"].nunique()

In [72]:
spredixcan_genes_n_models

gene_id
ENSG00000000419     2
ENSG00000000938    26
ENSG00000000971    34
ENSG00000001084    32
ENSG00000001167    40
                   ..
ENSG00000278540    36
ENSG00000278828     4
ENSG00000278845    49
ENSG00000281005    49
ENSG00000282608    36
Name: tissue, Length: 6425, dtype: int64

In [73]:
# testing that in S-MultiXcan I get the same number of tissues per gene
_tmp_smultixcan_results_n_models = (
    smultixcan_results.set_index("gene_id")["n"].astype(int).rename("tissue")
)

_cg = _tmp_smultixcan_results_n_models.index.intersection(
    spredixcan_genes_n_models.index
)
_tmp_smultixcan_results_n_models = _tmp_smultixcan_results_n_models.loc[_cg]
_spredixcan = spredixcan_genes_n_models.loc[_cg]

assert _spredixcan.shape[0] == _tmp_smultixcan_results_n_models.shape[0]
assert _spredixcan.equals(_tmp_smultixcan_results_n_models.loc[_spredixcan.index])

### Get tissues available per gene

In [74]:
spredixcan_genes_models = spredixcan_dfs.groupby("gene_id")["tissue"].apply(
    lambda x: frozenset(x.tolist())
)

In [75]:
spredixcan_genes_models

gene_id
ENSG00000000419         (Brain_Hypothalamus, Brain_Substantia_nigra)
ENSG00000000938    (Cells_EBV-transformed_lymphocytes, Brain_Ante...
ENSG00000000971    (Artery_Coronary, Brain_Anterior_cingulate_cor...
ENSG00000001084    (Artery_Coronary, Cells_EBV-transformed_lympho...
ENSG00000001167    (Artery_Coronary, Brain_Anterior_cingulate_cor...
                                         ...                        
ENSG00000278540    (Cells_EBV-transformed_lymphocytes, Brain_Ante...
ENSG00000278828    (Adipose_Visceral_Omentum, Artery_Coronary, Es...
ENSG00000278845    (Artery_Coronary, Cells_EBV-transformed_lympho...
ENSG00000281005    (Artery_Coronary, Cells_EBV-transformed_lympho...
ENSG00000282608    (Artery_Coronary, Brain_Anterior_cingulate_cor...
Name: tissue, Length: 6425, dtype: object

In [76]:
assert spredixcan_genes_n_models.shape[0] == spredixcan_genes_models.shape[0]

In [77]:
assert spredixcan_genes_n_models.index.equals(spredixcan_genes_models.index)

In [78]:
assert (spredixcan_genes_models.apply(len) <= len(prediction_model_tissues)).all()

In [79]:
spredixcan_genes_models.apply(len).describe()

count    6425.000000
mean       36.264591
std        12.946690
min         1.000000
25%        29.000000
50%        41.000000
75%        47.000000
max        49.000000
Name: tissue, dtype: float64

In [80]:
# testing
assert (
    spredixcan_genes_models.loc[spredixcan_genes_n_models.index]
    .apply(len)
    .equals(spredixcan_genes_n_models)
)

### Add gene name and set index

In [81]:
spredixcan_genes_models = spredixcan_genes_models.to_frame().reset_index()

In [82]:
spredixcan_genes_models.head()

,gene_id,tissue
0,ENSG00000000419,"(Brain_Hypothalamus, Brain_Substantia_nigra)"
1,ENSG00000000938,"(Cells_EBV-transformed_lymphocytes, Brain_Ante..."
2,ENSG00000000971,"(Artery_Coronary, Brain_Anterior_cingulate_cor..."
3,ENSG00000001084,"(Artery_Coronary, Cells_EBV-transformed_lympho..."
4,ENSG00000001167,"(Artery_Coronary, Brain_Anterior_cingulate_cor..."


In [83]:
spredixcan_genes_models = spredixcan_genes_models.assign(
    gene_name=spredixcan_genes_models["gene_id"].apply(
        lambda g: Gene.GENE_ID_TO_NAME_MAP[g]
    )
)

In [84]:
spredixcan_genes_models = spredixcan_genes_models[["gene_id", "gene_name", "tissue"]]

In [85]:
spredixcan_genes_models = spredixcan_genes_models.set_index("gene_id")

In [86]:
spredixcan_genes_models.head()

,gene_name,tissue
gene_id,,
ENSG00000000419,DPM1,"(Brain_Hypothalamus, Brain_Substantia_nigra)"
ENSG00000000938,FGR,"(Cells_EBV-transformed_lymphocytes, Brain_Ante..."
ENSG00000000971,CFH,"(Artery_Coronary, Brain_Anterior_cingulate_cor..."
ENSG00000001084,GCLC,"(Artery_Coronary, Cells_EBV-transformed_lympho..."
ENSG00000001167,NFYA,"(Artery_Coronary, Brain_Anterior_cingulate_cor..."


### Add number of tissues

In [87]:
spredixcan_genes_models = spredixcan_genes_models.assign(
    n_tissues=spredixcan_genes_models["tissue"].apply(len)
)

In [88]:
spredixcan_genes_models.head()

,gene_name,tissue,n_tissues
gene_id,,,
ENSG00000000419,DPM1,"(Brain_Hypothalamus, Brain_Substantia_nigra)",2
ENSG00000000938,FGR,"(Cells_EBV-transformed_lymphocytes, Brain_Ante...",26
ENSG00000000971,CFH,"(Artery_Coronary, Brain_Anterior_cingulate_cor...",34
ENSG00000001084,GCLC,"(Artery_Coronary, Cells_EBV-transformed_lympho...",32
ENSG00000001167,NFYA,"(Artery_Coronary, Brain_Anterior_cingulate_cor...",40


### Save

Here I quickly save these results to a file, given that the next steps (covariates) are slow to compute.

In [89]:
# this is important, other scripts depend on gene_name to be unique
assert spredixcan_genes_models["gene_name"].is_unique

In [90]:
assert not spredixcan_genes_models.isna().any(None)

In [91]:
spredixcan_genes_models.to_pickle(OUTPUT_DIR_BASE / "gene_tissues.pkl")

## Add covariates based on S-PrediXcan results

This extend the previous file with more columns

### Get gene's objects

In [92]:
spredixcan_gene_obj = {
    gene_id: Gene(ensembl_id=gene_id) for gene_id in spredixcan_genes_models.index
}

In [93]:
len(spredixcan_gene_obj)

6425

### Add genes' variance captured by principal components

In [94]:
def _get_gene_pc_variance(gene_row):
    gene_id = gene_row.name
    gene_tissues = gene_row["tissue"]
    gene_obj = spredixcan_gene_obj[gene_id]

    u, s, vt = gene_obj.get_tissues_correlations_svd(
        tissues=gene_tissues,
        snps_subset=gwas_variants_ids_set,
        reference_panel=REFERENCE_PANEL,
        model_type=EQTL_MODEL,
        # use_covariance_matrix=True,
    )

    return s

In [95]:
_tmp = spredixcan_genes_models.loc["ENSG00000188976"]
_get_gene_pc_variance(_tmp)

array([35.14301823,  3.87568728,  2.3091285 ,  1.90683623,  1.27573165])

In [96]:
spredixcan_genes_tissues_pc_variance = spredixcan_genes_models.apply(
    _get_gene_pc_variance, axis=1
)

In [97]:
spredixcan_genes_tissues_pc_variance

gene_id
ENSG00000000419             [1.0442398458252409, 0.9557601541747592]
ENSG00000000938    [19.95824749956773, 3.075997027819333, 2.08578...
ENSG00000000971    [21.56076992568329, 7.31017098704547, 1.837780...
ENSG00000001084    [21.291939080546527, 4.699418748743192, 2.2489...
ENSG00000001167              [37.33566029549769, 1.3914019604538372]
                                         ...                        
ENSG00000278540    [30.239540272954734, 3.044832646579891, 1.6793...
ENSG00000278828              [3.062182335109643, 0.9378176648903567]
ENSG00000278845              [45.687454510139084, 2.226801954648235]
ENSG00000281005                                 [48.301707397855715]
ENSG00000282608    [22.6535140812371, 6.245520030604538, 2.555945...
Length: 6425, dtype: object

In [98]:
# # testing
# assert spredixcan_genes_tissues_pc_variance.loc[
#     "ENSG00000188976"
# ].sum() == pytest.approx(44.01605629086847)
# # this is using the covariance:
# # assert spredixcan_genes_tissues_pc_variance.loc["ENSG00000188976"].sum() == pytest.approx(1.1492946006449425)

In [99]:
# add to spredixcan_genes_models
spredixcan_genes_models = spredixcan_genes_models.join(
    spredixcan_genes_tissues_pc_variance.rename("tissues_pc_variances")
)

In [100]:
spredixcan_genes_models.shape

(6425, 4)

In [101]:
spredixcan_genes_models.head()

,gene_name,tissue,n_tissues,tissues_pc_variances
gene_id,,,,
ENSG00000000419,DPM1,"(Brain_Hypothalamus, Brain_Substantia_nigra)",2,"[1.0442398458252409, 0.9557601541747592]"
ENSG00000000938,FGR,"(Cells_EBV-transformed_lymphocytes, Brain_Ante...",26,"[19.95824749956773, 3.075997027819333, 2.08578..."
ENSG00000000971,CFH,"(Artery_Coronary, Brain_Anterior_cingulate_cor...",34,"[21.56076992568329, 7.31017098704547, 1.837780..."
ENSG00000001084,GCLC,"(Artery_Coronary, Cells_EBV-transformed_lympho...",32,"[21.291939080546527, 4.699418748743192, 2.2489..."
ENSG00000001167,NFYA,"(Artery_Coronary, Brain_Anterior_cingulate_cor...",40,"[37.33566029549769, 1.3914019604538372]"


### Add gene variance per tissue

In [102]:
def _get_gene_variances(gene_row):
    gene_id = gene_row.name
    gene_tissues = gene_row["tissue"]

    tissue_variances = {}
    gene_obj = spredixcan_gene_obj[gene_id]

    for tissue in gene_tissues:
        tissue_var = gene_obj.get_pred_expression_variance(
            tissue=tissue,
            reference_panel=REFERENCE_PANEL,
            model_type=EQTL_MODEL,
            snps_subset=gwas_variants_ids_set,
        )

        if tissue_var is not None:
            tissue_variances[tissue] = tissue_var

    return tissue_variances

In [103]:
_tmp = spredixcan_genes_models.loc["ENSG00000000419"]
_get_gene_variances(_tmp)

{'Brain_Hypothalamus': 0.013162153504206677,
 'Brain_Substantia_nigra': 0.0004867380334260178}

In [104]:
spredixcan_genes_tissues_variance = spredixcan_genes_models.apply(
    _get_gene_variances, axis=1
)

In [105]:
spredixcan_genes_tissues_variance

gene_id
ENSG00000000419    {'Brain_Hypothalamus': 0.013162153504206677, '...
ENSG00000000938    {'Cells_EBV-transformed_lymphocytes': 0.003438...
ENSG00000000971    {'Artery_Coronary': 0.01500190292420916, 'Brai...
ENSG00000001084    {'Artery_Coronary': 1.0731972238129593e-06, 'C...
ENSG00000001167    {'Artery_Coronary': 0.028499385645099656, 'Bra...
                                         ...                        
ENSG00000278540    {'Cells_EBV-transformed_lymphocytes': 0.002975...
ENSG00000278828    {'Adipose_Visceral_Omentum': 0.000128291681544...
ENSG00000278845    {'Artery_Coronary': 0.01586217381279905, 'Cell...
ENSG00000281005    {'Artery_Coronary': 0.03502243595131216, 'Cell...
ENSG00000282608    {'Artery_Coronary': 0.00400244285687972, 'Brai...
Length: 6425, dtype: object

In [106]:
# # testing
# _gene_id = "ENSG00000188976"
# x = spredixcan_genes_tissues_variance.loc[_gene_id]
# # expected value obtained by sum of PCA eigenvalues on this gene's predicted expression
# assert np.sum(list(x.values())) == pytest.approx(1.2326202607409493)

In [107]:
# testing
spredixcan_genes_tissues_variance.loc["ENSG00000000419"]

{'Brain_Hypothalamus': 0.013162153504206677,
 'Brain_Substantia_nigra': 0.0004867380334260178}

In [108]:
# FIXME: maybe add more tests, these differt from GTEX V8
# # testing
# # here values were obtained from S-PrediXcan results, where the reference panel is GTEX V8, not 1000G, so just approximations)
# _gene_id = "ENSG00000000419"
# assert spredixcan_genes_tissues_variance.loc[_gene_id]["Brain_Substantia_nigra"] == pytest.approx(0.0004266255268163448)
# assert spredixcan_genes_tissues_variance.loc[_gene_id]["Brain_Hypothalamus"] == pytest.approx(0.011235877515236132)

In [109]:
# add to spredixcan_genes_models
spredixcan_genes_models = spredixcan_genes_models.join(
    spredixcan_genes_tissues_variance.rename("tissues_variances")
)

In [110]:
spredixcan_genes_models.shape

(6425, 5)

In [111]:
spredixcan_genes_models.head()

,gene_name,tissue,n_tissues,tissues_pc_variances,tissues_variances
gene_id,,,,,
ENSG00000000419,DPM1,"(Brain_Hypothalamus, Brain_Substantia_nigra)",2,"[1.0442398458252409, 0.9557601541747592]","{'Brain_Hypothalamus': 0.013162153504206677, '..."
ENSG00000000938,FGR,"(Cells_EBV-transformed_lymphocytes, Brain_Ante...",26,"[19.95824749956773, 3.075997027819333, 2.08578...",{'Cells_EBV-transformed_lymphocytes': 0.003438...
ENSG00000000971,CFH,"(Artery_Coronary, Brain_Anterior_cingulate_cor...",34,"[21.56076992568329, 7.31017098704547, 1.837780...","{'Artery_Coronary': 0.01500190292420916, 'Brai..."
ENSG00000001084,GCLC,"(Artery_Coronary, Cells_EBV-transformed_lympho...",32,"[21.291939080546527, 4.699418748743192, 2.2489...","{'Artery_Coronary': 1.0731972238129593e-06, 'C..."
ENSG00000001167,NFYA,"(Artery_Coronary, Brain_Anterior_cingulate_cor...",40,"[37.33566029549769, 1.3914019604538372]","{'Artery_Coronary': 0.028499385645099656, 'Bra..."


### Count number of SNPs predictors used across tissue models

In [112]:
spredixcan_genes_sum_of_n_snps_used = (
    spredixcan_dfs.groupby("gene_id")["n_snps_used"].sum().rename("n_snps_used_sum")
)

In [113]:
spredixcan_genes_sum_of_n_snps_used

gene_id
ENSG00000000419     2
ENSG00000000938    28
ENSG00000000971    44
ENSG00000001084    46
ENSG00000001167    47
                   ..
ENSG00000278540    44
ENSG00000278828     5
ENSG00000278845    89
ENSG00000281005    81
ENSG00000282608    40
Name: n_snps_used_sum, Length: 6425, dtype: int64

In [114]:
# add sum of snps used to spredixcan_genes_models
spredixcan_genes_models = spredixcan_genes_models.join(
    spredixcan_genes_sum_of_n_snps_used
)

In [115]:
spredixcan_genes_models.shape

(6425, 6)

In [116]:
spredixcan_genes_models.head()

,gene_name,tissue,n_tissues,tissues_pc_variances,tissues_variances,n_snps_used_sum
gene_id,,,,,,
ENSG00000000419,DPM1,"(Brain_Hypothalamus, Brain_Substantia_nigra)",2,"[1.0442398458252409, 0.9557601541747592]","{'Brain_Hypothalamus': 0.013162153504206677, '...",2
ENSG00000000938,FGR,"(Cells_EBV-transformed_lymphocytes, Brain_Ante...",26,"[19.95824749956773, 3.075997027819333, 2.08578...",{'Cells_EBV-transformed_lymphocytes': 0.003438...,28
ENSG00000000971,CFH,"(Artery_Coronary, Brain_Anterior_cingulate_cor...",34,"[21.56076992568329, 7.31017098704547, 1.837780...","{'Artery_Coronary': 0.01500190292420916, 'Brai...",44
ENSG00000001084,GCLC,"(Artery_Coronary, Cells_EBV-transformed_lympho...",32,"[21.291939080546527, 4.699418748743192, 2.2489...","{'Artery_Coronary': 1.0731972238129593e-06, 'C...",46
ENSG00000001167,NFYA,"(Artery_Coronary, Brain_Anterior_cingulate_cor...",40,"[37.33566029549769, 1.3914019604538372]","{'Artery_Coronary': 0.028499385645099656, 'Bra...",47


### Count number of SNPs predictors in models across tissue models

In [117]:
spredixcan_genes_sum_of_n_snps_in_model = (
    spredixcan_dfs.groupby("gene_id")["n_snps_in_model"]
    .sum()
    .rename("n_snps_in_model_sum")
)

In [118]:
spredixcan_genes_sum_of_n_snps_in_model

gene_id
ENSG00000000419     2
ENSG00000000938    30
ENSG00000000971    44
ENSG00000001084    46
ENSG00000001167    48
                   ..
ENSG00000278540    44
ENSG00000278828     5
ENSG00000278845    91
ENSG00000281005    81
ENSG00000282608    40
Name: n_snps_in_model_sum, Length: 6425, dtype: int64

In [119]:
# add sum of snps in model to spredixcan_genes_models
spredixcan_genes_models = spredixcan_genes_models.join(
    spredixcan_genes_sum_of_n_snps_in_model
)

In [120]:
spredixcan_genes_models.shape

(6425, 7)

In [121]:
spredixcan_genes_models.head()

,gene_name,tissue,n_tissues,tissues_pc_variances,tissues_variances,n_snps_used_sum,n_snps_in_model_sum
gene_id,,,,,,,
ENSG00000000419,DPM1,"(Brain_Hypothalamus, Brain_Substantia_nigra)",2,"[1.0442398458252409, 0.9557601541747592]","{'Brain_Hypothalamus': 0.013162153504206677, '...",2,2
ENSG00000000938,FGR,"(Cells_EBV-transformed_lymphocytes, Brain_Ante...",26,"[19.95824749956773, 3.075997027819333, 2.08578...",{'Cells_EBV-transformed_lymphocytes': 0.003438...,28,30
ENSG00000000971,CFH,"(Artery_Coronary, Brain_Anterior_cingulate_cor...",34,"[21.56076992568329, 7.31017098704547, 1.837780...","{'Artery_Coronary': 0.01500190292420916, 'Brai...",44,44
ENSG00000001084,GCLC,"(Artery_Coronary, Cells_EBV-transformed_lympho...",32,"[21.291939080546527, 4.699418748743192, 2.2489...","{'Artery_Coronary': 1.0731972238129593e-06, 'C...",46,46
ENSG00000001167,NFYA,"(Artery_Coronary, Brain_Anterior_cingulate_cor...",40,"[37.33566029549769, 1.3914019604538372]","{'Artery_Coronary': 0.028499385645099656, 'Bra...",47,48


### Summarize prediction models for each gene

In [122]:
def _summarize_gene_models(gene_id):
    """
    For a given gene ID, it returns a dataframe with predictor SNPs in rows and tissues in columns, where
    values are the weights of SNPs in those tissues.
    It can contain NaNs.
    """
    gene_obj = spredixcan_gene_obj[gene_id]
    gene_tissues = spredixcan_genes_models.loc[gene_id, "tissue"]

    gene_models = {}
    gene_unique_snps = set()
    for t in gene_tissues:
        gene_model = gene_obj.get_prediction_weights(tissue=t, model_type=EQTL_MODEL)
        gene_models[t] = gene_model

        gene_unique_snps.update(set(gene_model.index))

    df = pd.DataFrame(
        data=np.nan, index=list(gene_unique_snps), columns=list(gene_tissues)
    )

    for t in df.columns:
        for snp in df.index:
            gene_model = gene_models[t]

            if snp in gene_model.index:
                df.loc[snp, t] = gene_model.loc[snp]

    return df

In [123]:
# testing
spredixcan_gene_obj["ENSG00000000419"].get_prediction_weights(
    tissue="Brain_Hypothalamus", model_type=EQTL_MODEL
)

varID
chr20_50862947_C_T_b38    0.431375
Name: weight, dtype: float64

In [124]:
spredixcan_gene_obj["ENSG00000000419"].get_prediction_weights(
    tissue="Brain_Substantia_nigra", model_type=EQTL_MODEL
)

varID
chr20_50957480_C_T_b38   -0.146796
Name: weight, dtype: float64

In [125]:
# # testing
# _gene_id = "ENSG00000000419"

# _gene_model = _summarize_gene_models(_gene_id)
# assert (
#     _gene_model.loc["chr20_50862947_C_T_b38", "Brain_Hypothalamus"].round(5) == 0.43138
# )
# assert pd.isnull(_gene_model.loc["chr20_50957480_C_T_b38", "Brain_Hypothalamus"])

# assert pd.isnull(_gene_model.loc["chr20_50862947_C_T_b38", "Brain_Substantia_nigra"])
# assert (
#     _gene_model.loc["chr20_50957480_C_T_b38", "Brain_Substantia_nigra"].round(5)
#     == -0.1468
# )

In [126]:
gene_models = {}

for gene_id in spredixcan_genes_models.index:
    gene_models[gene_id] = _summarize_gene_models(gene_id)

In [127]:
# # testing
# _gene_id = "ENSG00000000419"

# _gene_model = gene_models[_gene_id]
# assert (
#     _gene_model.loc["chr20_50862947_C_T_b38", "Brain_Hypothalamus"].round(5) == 0.43138
# )
# assert pd.isnull(_gene_model.loc["chr20_50957480_C_T_b38", "Brain_Hypothalamus"])

# assert pd.isnull(_gene_model.loc["chr20_50862947_C_T_b38", "Brain_Substantia_nigra"])
# assert (
#     _gene_model.loc["chr20_50957480_C_T_b38", "Brain_Substantia_nigra"].round(5)
#     == -0.1468
# )

In [128]:
# save
import gzip

with gzip.GzipFile(OUTPUT_DIR_BASE / "gene_tissues_models.pkl.gz", "w") as f:
    pickle.dump(gene_models, f)

In [129]:
# testing saved file
with gzip.GzipFile(OUTPUT_DIR_BASE / "gene_tissues_models.pkl.gz", "r") as f:
    _tmp = pickle.load(f)

In [130]:
assert len(gene_models) == len(_tmp)
assert gene_models["ENSG00000000419"].equals(_tmp["ENSG00000000419"])

### Count number of _unique_ SNPs predictors used and available across tissue models

In [131]:
def _count_unique_snps(gene_id):
    """
    For a gene_id, it counts unique SNPs in all models and their intersection with GWAS SNPs (therefore, used by S-PrediXcan).
    """
    gene_tissues = spredixcan_genes_models.loc[gene_id, "tissue"]

    gene_unique_snps = set()
    for t in gene_tissues:
        t_snps = set(gene_models[gene_id].index)
        gene_unique_snps.update(t_snps)

    gene_unique_snps_in_gwas = gwas_variants_ids_set.intersection(gene_unique_snps)

    return pd.Series(
        {
            "unique_n_snps_in_model": len(gene_unique_snps),
            "unique_n_snps_used": len(gene_unique_snps_in_gwas),
        }
    )

In [132]:
# testing
spredixcan_genes_models[spredixcan_genes_models["n_snps_used_sum"] == 2].head()

,gene_name,tissue,n_tissues,tissues_pc_variances,tissues_variances,n_snps_used_sum,n_snps_in_model_sum
gene_id,,,,,,,
ENSG00000000419,DPM1,"(Brain_Hypothalamus, Brain_Substantia_nigra)",2,"[1.0442398458252409, 0.9557601541747592]","{'Brain_Hypothalamus': 0.013162153504206677, '...",2,2
ENSG00000010256,UQCRC1,"(Thyroid, Whole_Blood)",2,"[1.1048350039574655, 0.8951649960425345]","{'Thyroid': 0.0008025245652862433, 'Whole_Bloo...",2,2
ENSG00000017427,IGF1,"(Brain_Amygdala, Testis)",2,"[1.0389319526404415, 0.9610680473595585]","{'Brain_Amygdala': 0.02059332044751194, 'Testi...",2,2
ENSG00000043093,DCUN1D1,"(Esophagus_Gastroesophageal_Junction, Esophagu...",2,"[1.4951653523020276, 0.5048346476979725]",{'Esophagus_Gastroesophageal_Junction': 0.0002...,2,2
ENSG00000081377,CDC14B,"(Muscle_Skeletal, Brain_Nucleus_accumbens_basa...",2,"[1.1285007973441137, 0.8714992026558863]","{'Muscle_Skeletal': 0.0076846621297156385, 'Br...",2,2


In [133]:
# case with two snps, not repeated across tissues
_gene_id = "ENSG00000000419"
display(
    spredixcan_gene_obj[_gene_id].get_prediction_weights(
        tissue="Brain_Hypothalamus", model_type=EQTL_MODEL
    )
)
display(
    spredixcan_gene_obj[_gene_id].get_prediction_weights(
        tissue="Brain_Substantia_nigra", model_type=EQTL_MODEL
    )
)

varID
chr20_50862947_C_T_b38    0.431375
Name: weight, dtype: float64

varID
chr20_50957480_C_T_b38   -0.146796
Name: weight, dtype: float64

In [134]:
# _tmp = _count_unique_snps(_gene_id)
# assert _tmp.shape[0] == 2
# assert _tmp["unique_n_snps_in_model"] == 2
# assert _tmp["unique_n_snps_used"] == 2

In [135]:
# get unique snps for all genes
spredixcan_genes_unique_n_snps = spredixcan_genes_models.groupby("gene_id").apply(
    lambda x: _count_unique_snps(x.name)
)

In [136]:
spredixcan_genes_unique_n_snps.head()

,unique_n_snps_in_model,unique_n_snps_used
gene_id,,
ENSG00000000419,2,2
ENSG00000000938,5,4
ENSG00000000971,12,12
ENSG00000001084,23,23
ENSG00000001167,14,13


In [137]:
assert (
    spredixcan_genes_unique_n_snps["unique_n_snps_in_model"]
    >= spredixcan_genes_unique_n_snps["unique_n_snps_used"]
).all()

In [138]:
# add unique snps to spredixcan_genes_models
spredixcan_genes_models = spredixcan_genes_models.join(spredixcan_genes_unique_n_snps)

In [139]:
spredixcan_genes_models.shape

(6425, 9)

In [140]:
spredixcan_genes_models.head()

,gene_name,tissue,n_tissues,tissues_pc_variances,tissues_variances,n_snps_used_sum,n_snps_in_model_sum,unique_n_snps_in_model,unique_n_snps_used
gene_id,,,,,,,,,
ENSG00000000419,DPM1,"(Brain_Hypothalamus, Brain_Substantia_nigra)",2,"[1.0442398458252409, 0.9557601541747592]","{'Brain_Hypothalamus': 0.013162153504206677, '...",2,2,2,2
ENSG00000000938,FGR,"(Cells_EBV-transformed_lymphocytes, Brain_Ante...",26,"[19.95824749956773, 3.075997027819333, 2.08578...",{'Cells_EBV-transformed_lymphocytes': 0.003438...,28,30,5,4
ENSG00000000971,CFH,"(Artery_Coronary, Brain_Anterior_cingulate_cor...",34,"[21.56076992568329, 7.31017098704547, 1.837780...","{'Artery_Coronary': 0.01500190292420916, 'Brai...",44,44,12,12
ENSG00000001084,GCLC,"(Artery_Coronary, Cells_EBV-transformed_lympho...",32,"[21.291939080546527, 4.699418748743192, 2.2489...","{'Artery_Coronary': 1.0731972238129593e-06, 'C...",46,46,23,23
ENSG00000001167,NFYA,"(Artery_Coronary, Brain_Anterior_cingulate_cor...",40,"[37.33566029549769, 1.3914019604538372]","{'Artery_Coronary': 0.028499385645099656, 'Bra...",47,48,14,13


### Save

In [141]:
# this is important, other scripts depend on gene_name to be unique
assert spredixcan_genes_models["gene_name"].is_unique

In [142]:
assert not spredixcan_genes_models.isna().any(None)

In [143]:
spredixcan_genes_models.to_pickle(OUTPUT_DIR_BASE / "gene_tissues.pkl")